<a href="https://colab.research.google.com/github/minddo/affective-computing/blob/master/2_DEAP_Test_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load data & import package

In [ ]:
# mount google drive 
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pickle
dataset=pickle.load(open('/content/drive/My Drive/DEAP/dataset.pkl', 'rb'), encoding='iso-8859-1')

In [ ]:
df=dataset['s01']

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Activation
from tensorflow.keras import layers
from keras.layers.normalization import BatchNormalization
from keras.layers import Bidirectional
from functools import partial
from keras.callbacks import ModelCheckpoint
from keras.layers import Conv2D, MaxPooling2D

In [ ]:
from keras import backend as K

def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

#Model-bidriectional LSTM

In [ ]:
def model_work_L(size, X, Y):
    model = Sequential()
    model.add(Bidirectional(LSTM(128, return_sequences=True, dropout=0.2, recurrent_dropout=0.2), input_shape=(10,128*size)))
    model.add(BatchNormalization())
    model.add(Bidirectional(LSTM(128, dropout=0.2, recurrent_dropout=0.2)))
    model.add(BatchNormalization())

    model.add(Dense(2, activation='softmax'))





    from sklearn.model_selection import KFold
    acc=[]
    f=[]
   
    n_split=3
    
    for train_index,test_index in KFold(n_split).split(X):
        x_train,x_test=X[train_index],X[test_index]
        y_train,y_test=Y[train_index],Y[test_index]
        model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy",f1_m])
        early_stopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
        model.fit(x_train, y_train,epochs=100 ,validation_data=( x_test, y_test), callbacks=[early_stopping_cb])
        print('Model evaluation ',model.evaluate(x_test,y_test))
        acc.append(model.evaluate(x_test,y_test)[1] * 100)
        f.append(model.evaluate(x_test,y_test)[2] * 100)
   
    print("%.2f%% (+/- %.2f%%)" % (np.mean(acc), np.std(acc)))
    print("%.2f%% (+/- %.2f%%)" % (np.mean(f), np.std(f)))

## Arousal

### GSR

In [ ]:
size=1
X=df['data']['GSR']
Y=df['labels']['arousal']

model_work_L(size)

Epoch 1/100
5/5 [==============================] - 1s 219ms/step - loss: 0.9706 - accuracy: 0.5125 - f1_m: 0.5125 - val_loss: 0.5926 - val_accuracy: 0.7000 - val_f1_m: 0.7500
Epoch 2/100
5/5 [==============================] - 0s 56ms/step - loss: 0.9912 - accuracy: 0.5688 - f1_m: 0.5687 - val_loss: 0.6057 - val_accuracy: 0.7000 - val_f1_m: 0.7500
Epoch 3/100
5/5 [==============================] - 0s 58ms/step - loss: 0.9114 - accuracy: 0.5875 - f1_m: 0.5875 - val_loss: 0.6385 - val_accuracy: 0.7250 - val_f1_m: 0.7708
Epoch 4/100
5/5 [==============================] - 0s 57ms/step - loss: 0.8237 - accuracy: 0.5875 - f1_m: 0.5875 - val_loss: 0.6034 - val_accuracy: 0.7000 - val_f1_m: 0.7500
Epoch 5/100
5/5 [==============================] - 0s 56ms/step - loss: 0.9327 - accuracy: 0.5562 - f1_m: 0.5562 - val_loss: 0.6134 - val_accuracy: 0.7250 - val_f1_m: 0.7708
Epoch 6/100
5/5 [==============================] - 0s 58ms/step - loss: 0.7978 - accuracy: 0.6187 - f1_m: 0.6187 - val_loss: 0.65

### PPG

In [ ]:
X=df['data']['PPG']
Y=df['labels']['arousal']

model_work_L(size)

Epoch 1/100
5/5 [==============================] - 1s 222ms/step - loss: 1.2447 - accuracy: 0.5063 - f1_m: 0.5062 - val_loss: 0.6727 - val_accuracy: 0.6375 - val_f1_m: 0.6979
Epoch 2/100
5/5 [==============================] - 0s 56ms/step - loss: 0.9280 - accuracy: 0.6062 - f1_m: 0.6062 - val_loss: 0.7101 - val_accuracy: 0.3000 - val_f1_m: 0.2500
Epoch 3/100
5/5 [==============================] - 0s 58ms/step - loss: 0.9719 - accuracy: 0.5188 - f1_m: 0.5187 - val_loss: 0.7410 - val_accuracy: 0.3000 - val_f1_m: 0.2500
Epoch 4/100
5/5 [==============================] - 0s 64ms/step - loss: 0.8065 - accuracy: 0.6062 - f1_m: 0.6062 - val_loss: 0.7278 - val_accuracy: 0.3000 - val_f1_m: 0.2500
Epoch 5/100
5/5 [==============================] - 0s 60ms/step - loss: 0.7230 - accuracy: 0.6250 - f1_m: 0.6250 - val_loss: 0.7731 - val_accuracy: 0.3000 - val_f1_m: 0.2500
Epoch 6/100
5/5 [==============================] - 0s 60ms/step - loss: 0.7226 - accuracy: 0.6062 - f1_m: 0.6062 - val_loss: 0.72

### Temperature

In [ ]:
X=df['data']['TEMP']
Y=df['labels']['arousal']

model_work_L(size)

Epoch 1/100
5/5 [==============================] - 1s 219ms/step - loss: 1.0561 - accuracy: 0.4875 - f1_m: 0.4875 - val_loss: 0.8012 - val_accuracy: 0.3000 - val_f1_m: 0.2500
Epoch 2/100
5/5 [==============================] - 0s 60ms/step - loss: 1.0451 - accuracy: 0.5063 - f1_m: 0.5062 - val_loss: 0.7541 - val_accuracy: 0.3000 - val_f1_m: 0.2500
Epoch 3/100
5/5 [==============================] - 0s 57ms/step - loss: 0.9694 - accuracy: 0.5562 - f1_m: 0.5562 - val_loss: 0.8783 - val_accuracy: 0.3000 - val_f1_m: 0.2500
Epoch 4/100
5/5 [==============================] - 0s 56ms/step - loss: 0.8268 - accuracy: 0.5500 - f1_m: 0.5500 - val_loss: 0.8041 - val_accuracy: 0.3000 - val_f1_m: 0.2500
Epoch 5/100
5/5 [==============================] - 0s 56ms/step - loss: 0.8143 - accuracy: 0.6062 - f1_m: 0.6062 - val_loss: 0.9724 - val_accuracy: 0.3000 - val_f1_m: 0.2500
Epoch 6/100
5/5 [==============================] - 0s 56ms/step - loss: 0.7134 - accuracy: 0.6687 - f1_m: 0.6687 - val_loss: 0.92

### GSR+PPG

In [ ]:
size=2
X=np.concatenate((df['data']['GSR'],df['data']['PPG'],),axis=2)
Y=df['labels']['arousal']

model_work_L(size)

Epoch 1/100
5/5 [==============================] - 1s 221ms/step - loss: 0.9606 - accuracy: 0.5688 - f1_m: 0.5687 - val_loss: 0.5979 - val_accuracy: 0.7000 - val_f1_m: 0.7500
Epoch 2/100
5/5 [==============================] - 0s 56ms/step - loss: 0.8729 - accuracy: 0.6125 - f1_m: 0.6125 - val_loss: 0.6003 - val_accuracy: 0.7000 - val_f1_m: 0.7500
Epoch 3/100
5/5 [==============================] - 0s 59ms/step - loss: 0.9138 - accuracy: 0.5562 - f1_m: 0.5562 - val_loss: 0.5986 - val_accuracy: 0.7000 - val_f1_m: 0.7500
Epoch 4/100
5/5 [==============================] - 0s 56ms/step - loss: 0.8090 - accuracy: 0.5938 - f1_m: 0.5937 - val_loss: 0.6075 - val_accuracy: 0.7000 - val_f1_m: 0.7500
Epoch 5/100
5/5 [==============================] - 0s 57ms/step - loss: 0.8359 - accuracy: 0.5813 - f1_m: 0.5812 - val_loss: 0.6012 - val_accuracy: 0.7000 - val_f1_m: 0.7500
Epoch 6/100
5/5 [==============================] - 0s 61ms/step - loss: 0.6901 - accuracy: 0.6500 - f1_m: 0.6500 - val_loss: 0.64

### GSR+Temperature

In [ ]:
X=np.concatenate((df['data']['GSR'],df['data']['TEMP'],),axis=2)
Y=df['labels']['arousal']

model_work_L(size)

Epoch 1/100
5/5 [==============================] - 1s 221ms/step - loss: 1.1451 - accuracy: 0.4625 - f1_m: 0.4625 - val_loss: 0.6112 - val_accuracy: 0.7000 - val_f1_m: 0.7500
Epoch 2/100
5/5 [==============================] - 0s 58ms/step - loss: 0.7809 - accuracy: 0.6375 - f1_m: 0.6375 - val_loss: 0.5970 - val_accuracy: 0.7000 - val_f1_m: 0.7500
Epoch 3/100
5/5 [==============================] - 0s 58ms/step - loss: 0.9779 - accuracy: 0.5125 - f1_m: 0.5125 - val_loss: 0.6058 - val_accuracy: 0.7000 - val_f1_m: 0.7500
Epoch 4/100
5/5 [==============================] - 0s 60ms/step - loss: 0.8103 - accuracy: 0.5813 - f1_m: 0.5812 - val_loss: 0.5939 - val_accuracy: 0.7000 - val_f1_m: 0.7500
Epoch 5/100
5/5 [==============================] - 0s 56ms/step - loss: 0.7183 - accuracy: 0.6313 - f1_m: 0.6312 - val_loss: 0.6453 - val_accuracy: 0.7375 - val_f1_m: 0.7812
Epoch 6/100
5/5 [==============================] - 0s 58ms/step - loss: 0.7675 - accuracy: 0.5688 - f1_m: 0.5687 - val_loss: 0.82

### PPG+Temperature

In [ ]:
X=np.concatenate((df['data']['PPG'],df['data']['TEMP'],),axis=2)
Y=df['labels']['arousal']

model_work_L(size)

Epoch 1/100
5/5 [==============================] - 1s 218ms/step - loss: 1.4617 - accuracy: 0.5125 - f1_m: 0.5125 - val_loss: 0.6353 - val_accuracy: 0.6750 - val_f1_m: 0.7292
Epoch 2/100
5/5 [==============================] - 0s 57ms/step - loss: 0.9451 - accuracy: 0.6187 - f1_m: 0.6187 - val_loss: 0.6466 - val_accuracy: 0.7000 - val_f1_m: 0.7500
Epoch 3/100
5/5 [==============================] - 0s 58ms/step - loss: 0.8377 - accuracy: 0.5750 - f1_m: 0.5750 - val_loss: 0.7243 - val_accuracy: 0.3500 - val_f1_m: 0.3125
Epoch 4/100
5/5 [==============================] - 0s 57ms/step - loss: 0.8056 - accuracy: 0.5625 - f1_m: 0.5625 - val_loss: 0.7168 - val_accuracy: 0.3000 - val_f1_m: 0.2500
Epoch 5/100
5/5 [==============================] - 0s 57ms/step - loss: 0.7447 - accuracy: 0.5938 - f1_m: 0.5937 - val_loss: 0.7230 - val_accuracy: 0.3375 - val_f1_m: 0.2917
Epoch 6/100
5/5 [==============================] - 0s 59ms/step - loss: 0.7147 - accuracy: 0.6313 - f1_m: 0.6312 - val_loss: 0.78

###All

In [ ]:
size=3
X=np.concatenate((df['data']['GSR'],df['data']['PPG'],df['data']['TEMP'],),axis=2)
Y=df['labels']['arousal']

model_work_L(size)

Epoch 1/100
5/5 [==============================] - 1s 227ms/step - loss: 1.0512 - accuracy: 0.5500 - f1_m: 0.5500 - val_loss: 0.6099 - val_accuracy: 0.7000 - val_f1_m: 0.7500
Epoch 2/100
5/5 [==============================] - 0s 59ms/step - loss: 1.0518 - accuracy: 0.5125 - f1_m: 0.5125 - val_loss: 0.6049 - val_accuracy: 0.7000 - val_f1_m: 0.7500
Epoch 3/100
5/5 [==============================] - 0s 57ms/step - loss: 0.8414 - accuracy: 0.5375 - f1_m: 0.5375 - val_loss: 0.6086 - val_accuracy: 0.7000 - val_f1_m: 0.7500
Epoch 4/100
5/5 [==============================] - 0s 55ms/step - loss: 0.8273 - accuracy: 0.5437 - f1_m: 0.5437 - val_loss: 0.6167 - val_accuracy: 0.7000 - val_f1_m: 0.7500
Epoch 5/100
5/5 [==============================] - 0s 60ms/step - loss: 0.7900 - accuracy: 0.5688 - f1_m: 0.5687 - val_loss: 0.6497 - val_accuracy: 0.7250 - val_f1_m: 0.7604
Epoch 6/100
5/5 [==============================] - 0s 55ms/step - loss: 0.7006 - accuracy: 0.6125 - f1_m: 0.6125 - val_loss: 0.63

##Valence

### GSR

In [ ]:
size=1
X=df['data']['GSR']
Y=df['labels']['valence']

model_work_L(size)

Epoch 1/100
5/5 [==============================] - 1s 223ms/step - loss: 1.0162 - accuracy: 0.4688 - f1_m: 0.4687 - val_loss: 0.7696 - val_accuracy: 0.4750 - val_f1_m: 0.5625
Epoch 2/100
5/5 [==============================] - 0s 59ms/step - loss: 1.0256 - accuracy: 0.5750 - f1_m: 0.5750 - val_loss: 0.8002 - val_accuracy: 0.4750 - val_f1_m: 0.5625
Epoch 3/100
5/5 [==============================] - 0s 59ms/step - loss: 0.9611 - accuracy: 0.5813 - f1_m: 0.5812 - val_loss: 0.8007 - val_accuracy: 0.4750 - val_f1_m: 0.5625
Epoch 4/100
5/5 [==============================] - 0s 59ms/step - loss: 0.9497 - accuracy: 0.5625 - f1_m: 0.5625 - val_loss: 0.7745 - val_accuracy: 0.4750 - val_f1_m: 0.5625
Epoch 5/100
5/5 [==============================] - 0s 57ms/step - loss: 0.9696 - accuracy: 0.5188 - f1_m: 0.5187 - val_loss: 0.7710 - val_accuracy: 0.4750 - val_f1_m: 0.5625
Epoch 6/100
5/5 [==============================] - 0s 62ms/step - loss: 0.8780 - accuracy: 0.5562 - f1_m: 0.5562 - val_loss: 0.74

### PPG

In [ ]:

X=df['data']['PPG']
Y=df['labels']['valence']

model_work_L(size)

Epoch 1/100
5/5 [==============================] - 1s 219ms/step - loss: 1.2931 - accuracy: 0.5063 - f1_m: 0.5062 - val_loss: 0.7261 - val_accuracy: 0.4750 - val_f1_m: 0.5625
Epoch 2/100
5/5 [==============================] - 0s 56ms/step - loss: 1.0044 - accuracy: 0.5500 - f1_m: 0.5500 - val_loss: 0.7172 - val_accuracy: 0.4375 - val_f1_m: 0.3646
Epoch 3/100
5/5 [==============================] - 0s 59ms/step - loss: 1.0715 - accuracy: 0.5375 - f1_m: 0.5375 - val_loss: 0.7491 - val_accuracy: 0.4750 - val_f1_m: 0.5625
Epoch 4/100
5/5 [==============================] - 0s 56ms/step - loss: 0.9156 - accuracy: 0.4938 - f1_m: 0.4937 - val_loss: 0.7431 - val_accuracy: 0.4750 - val_f1_m: 0.5625
Epoch 5/100
5/5 [==============================] - 0s 59ms/step - loss: 0.8662 - accuracy: 0.5375 - f1_m: 0.5375 - val_loss: 0.7401 - val_accuracy: 0.4750 - val_f1_m: 0.5625
Epoch 6/100
5/5 [==============================] - 0s 59ms/step - loss: 0.8533 - accuracy: 0.5375 - f1_m: 0.5375 - val_loss: 0.74

### Temperature

In [ ]:

X=df['data']['TEMP']
Y=df['labels']['valence']

model_work_L(size)

Epoch 1/100
5/5 [==============================] - 1s 221ms/step - loss: 1.1425 - accuracy: 0.4938 - f1_m: 0.4937 - val_loss: 0.7358 - val_accuracy: 0.4750 - val_f1_m: 0.5625
Epoch 2/100
5/5 [==============================] - 0s 57ms/step - loss: 0.9080 - accuracy: 0.5312 - f1_m: 0.5312 - val_loss: 0.8219 - val_accuracy: 0.4750 - val_f1_m: 0.5625
Epoch 3/100
5/5 [==============================] - 0s 58ms/step - loss: 0.9940 - accuracy: 0.5125 - f1_m: 0.5125 - val_loss: 0.7581 - val_accuracy: 0.4750 - val_f1_m: 0.5625
Epoch 4/100
5/5 [==============================] - 0s 55ms/step - loss: 0.9855 - accuracy: 0.4750 - f1_m: 0.4750 - val_loss: 0.7636 - val_accuracy: 0.4750 - val_f1_m: 0.5625
Epoch 5/100
5/5 [==============================] - 0s 57ms/step - loss: 0.9782 - accuracy: 0.5125 - f1_m: 0.5125 - val_loss: 0.7369 - val_accuracy: 0.4750 - val_f1_m: 0.5625
Epoch 6/100
5/5 [==============================] - 0s 59ms/step - loss: 0.8697 - accuracy: 0.5375 - f1_m: 0.5375 - val_loss: 0.71

### GSR+PPG

In [ ]:
size=2
X=np.concatenate((df['data']['GSR'],df['data']['PPG'],),axis=2)
Y=df['labels']['valence']

model_work_L(size)

Epoch 1/100
5/5 [==============================] - 1s 216ms/step - loss: 1.0891 - accuracy: 0.5938 - f1_m: 0.5937 - val_loss: 0.9367 - val_accuracy: 0.4750 - val_f1_m: 0.5625
Epoch 2/100
5/5 [==============================] - 0s 56ms/step - loss: 0.9807 - accuracy: 0.5437 - f1_m: 0.5437 - val_loss: 0.8744 - val_accuracy: 0.4750 - val_f1_m: 0.5625
Epoch 3/100
5/5 [==============================] - 0s 57ms/step - loss: 0.9012 - accuracy: 0.5750 - f1_m: 0.5750 - val_loss: 0.8605 - val_accuracy: 0.4750 - val_f1_m: 0.5625
Epoch 4/100
5/5 [==============================] - 0s 59ms/step - loss: 0.9078 - accuracy: 0.5688 - f1_m: 0.5687 - val_loss: 0.8491 - val_accuracy: 0.4750 - val_f1_m: 0.5625
Epoch 5/100
5/5 [==============================] - 0s 59ms/step - loss: 0.8508 - accuracy: 0.5813 - f1_m: 0.5812 - val_loss: 0.8899 - val_accuracy: 0.4750 - val_f1_m: 0.5625
Epoch 6/100
5/5 [==============================] - 0s 56ms/step - loss: 0.7647 - accuracy: 0.6438 - f1_m: 0.6437 - val_loss: 0.99

### GSR+Temperature

In [ ]:
X=np.concatenate((df['data']['GSR'],df['data']['TEMP'],),axis=2)
Y=df['labels']['valence']

model_work_L(size)

Epoch 1/100
5/5 [==============================] - 1s 216ms/step - loss: 0.8958 - accuracy: 0.5750 - f1_m: 0.5750 - val_loss: 0.9369 - val_accuracy: 0.4750 - val_f1_m: 0.5625
Epoch 2/100
5/5 [==============================] - 0s 61ms/step - loss: 1.0163 - accuracy: 0.5312 - f1_m: 0.5312 - val_loss: 0.9911 - val_accuracy: 0.4750 - val_f1_m: 0.5625
Epoch 3/100
5/5 [==============================] - 0s 54ms/step - loss: 0.8054 - accuracy: 0.5813 - f1_m: 0.5812 - val_loss: 0.9408 - val_accuracy: 0.4750 - val_f1_m: 0.5625
Epoch 4/100
5/5 [==============================] - 0s 57ms/step - loss: 0.8418 - accuracy: 0.5688 - f1_m: 0.5687 - val_loss: 0.8624 - val_accuracy: 0.4750 - val_f1_m: 0.5625
Epoch 5/100
5/5 [==============================] - 0s 58ms/step - loss: 0.8510 - accuracy: 0.5312 - f1_m: 0.5312 - val_loss: 0.8760 - val_accuracy: 0.4750 - val_f1_m: 0.5625
Epoch 6/100
5/5 [==============================] - 0s 55ms/step - loss: 0.8221 - accuracy: 0.5375 - f1_m: 0.5375 - val_loss: 0.88

### PPG+Temperature

In [ ]:
X=np.concatenate((df['data']['PPG'],df['data']['TEMP'],),axis=2)
Y=df['labels']['valence']

model_work_L(size)

Epoch 1/100
5/5 [==============================] - 1s 217ms/step - loss: 0.8870 - accuracy: 0.5688 - f1_m: 0.5687 - val_loss: 0.7112 - val_accuracy: 0.5250 - val_f1_m: 0.4375
Epoch 2/100
5/5 [==============================] - 0s 58ms/step - loss: 1.0412 - accuracy: 0.5500 - f1_m: 0.5500 - val_loss: 0.7026 - val_accuracy: 0.4750 - val_f1_m: 0.5625
Epoch 3/100
5/5 [==============================] - 0s 58ms/step - loss: 1.0308 - accuracy: 0.5063 - f1_m: 0.5062 - val_loss: 0.6874 - val_accuracy: 0.6375 - val_f1_m: 0.6042
Epoch 4/100
5/5 [==============================] - 0s 58ms/step - loss: 0.9406 - accuracy: 0.4750 - f1_m: 0.4750 - val_loss: 0.6845 - val_accuracy: 0.5500 - val_f1_m: 0.4896
Epoch 5/100
5/5 [==============================] - 0s 57ms/step - loss: 0.9110 - accuracy: 0.5000 - f1_m: 0.5000 - val_loss: 0.7127 - val_accuracy: 0.4750 - val_f1_m: 0.5625
Epoch 6/100
5/5 [==============================] - 0s 55ms/step - loss: 1.0229 - accuracy: 0.5125 - f1_m: 0.5125 - val_loss: 0.71

### All

In [ ]:
size=3
X=np.concatenate((df['data']['GSR'],df['data']['PPG'],df['data']['TEMP'],),axis=2)
Y=df['labels']['valence']

model_work_L(size)

Epoch 1/100
5/5 [==============================] - 1s 220ms/step - loss: 0.9982 - accuracy: 0.5375 - f1_m: 0.5375 - val_loss: 0.7508 - val_accuracy: 0.4750 - val_f1_m: 0.5625
Epoch 2/100
5/5 [==============================] - 0s 55ms/step - loss: 0.9338 - accuracy: 0.5562 - f1_m: 0.5562 - val_loss: 0.7587 - val_accuracy: 0.4750 - val_f1_m: 0.5625
Epoch 3/100
5/5 [==============================] - 0s 54ms/step - loss: 0.8706 - accuracy: 0.5938 - f1_m: 0.5937 - val_loss: 0.8117 - val_accuracy: 0.4750 - val_f1_m: 0.5625
Epoch 4/100
5/5 [==============================] - 0s 60ms/step - loss: 0.8546 - accuracy: 0.5625 - f1_m: 0.5625 - val_loss: 0.7917 - val_accuracy: 0.4750 - val_f1_m: 0.5625
Epoch 5/100
5/5 [==============================] - 0s 57ms/step - loss: 0.7131 - accuracy: 0.6313 - f1_m: 0.6312 - val_loss: 0.7670 - val_accuracy: 0.4750 - val_f1_m: 0.5625
Epoch 6/100
5/5 [==============================] - 0s 55ms/step - loss: 0.7390 - accuracy: 0.6500 - f1_m: 0.6500 - val_loss: 0.78

# Model-CRNN

In [ ]:
def model_work(size):
    model = Sequential()
    model.add(Conv2D(64, (3, 3), padding='same', name='conv1', kernel_initializer='he_normal', input_shape=(10, 128, size)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), name='max1'))

    model.add(Conv2D(128, (3, 3), padding='same', name='conv2', kernel_initializer='he_normal'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))



    model.add(keras.layers.Reshape((10,32*128,) ))
    model.add(Dense(64, activation='relu', kernel_initializer='he_normal', name='dense1'))
    model.add(Bidirectional(LSTM(128, return_sequences=True, dropout=0.2, recurrent_dropout=0.2), input_shape=(10,128)))
    model.add(BatchNormalization())
    model.add(Bidirectional(LSTM(128, dropout=0.2, recurrent_dropout=0.2)))
    model.add(BatchNormalization())

    model.add(Dense(2, activation='softmax'))




    from sklearn.model_selection import KFold
    acc=[]
    f=[]
   
    n_split=3
    
    for train_index,test_index in KFold(n_split).split(X):
        x_train,x_test=X[train_index],X[test_index]
        y_train,y_test=Y[train_index],Y[test_index]
        model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy",f1_m])
        early_stopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
        model.fit(x_train, y_train,epochs=100 ,validation_data=( x_test, y_test), callbacks=[early_stopping_cb])
        print('Model evaluation ',model.evaluate(x_test,y_test))
        acc.append(model.evaluate(x_test,y_test)[1] * 100)
        f.append(model.evaluate(x_test,y_test)[2] * 100)
   
    print("%.2f%% (+/- %.2f%%)" % (np.mean(acc), np.std(acc)))
    print("%.2f%% (+/- %.2f%%)" % (np.mean(f), np.std(f)))

## Arousal

### GSR

In [ ]:
size=1
X=df['data']['GSR'].reshape(-1,10,128,size)
Y=df['labels']['arousal']

model_work(size)

Epoch 1/100
5/5 [==============================] - 1s 248ms/step - loss: 0.9151 - accuracy: 0.5188 - f1_m: 0.5187 - val_loss: 0.6876 - val_accuracy: 0.7000 - val_f1_m: 0.7500
Epoch 2/100
5/5 [==============================] - 0s 67ms/step - loss: 0.9388 - accuracy: 0.5312 - f1_m: 0.5312 - val_loss: 0.6688 - val_accuracy: 0.7000 - val_f1_m: 0.7500
Epoch 3/100
5/5 [==============================] - 0s 68ms/step - loss: 0.9228 - accuracy: 0.5188 - f1_m: 0.5187 - val_loss: 0.6460 - val_accuracy: 0.7000 - val_f1_m: 0.7500
Epoch 4/100
5/5 [==============================] - 0s 66ms/step - loss: 0.7648 - accuracy: 0.6125 - f1_m: 0.6125 - val_loss: 0.6559 - val_accuracy: 0.7250 - val_f1_m: 0.7708
Epoch 5/100
5/5 [==============================] - 0s 63ms/step - loss: 0.7269 - accuracy: 0.6375 - f1_m: 0.6375 - val_loss: 0.6463 - val_accuracy: 0.7000 - val_f1_m: 0.7500
Epoch 6/100
5/5 [==============================] - 0s 64ms/step - loss: 0.7950 - accuracy: 0.5938 - f1_m: 0.5937 - val_loss: 0.67

### PPG

In [ ]:
size=1
X=df['data']['PPG'].reshape(-1,10,128,size)
Y=df['labels']['arousal']

model_work(1)

Epoch 1/100
5/5 [==============================] - 1s 237ms/step - loss: 1.2732 - accuracy: 0.4750 - f1_m: 0.4750 - val_loss: 0.8516 - val_accuracy: 0.3000 - val_f1_m: 0.2500
Epoch 2/100
5/5 [==============================] - 0s 61ms/step - loss: 0.9299 - accuracy: 0.5125 - f1_m: 0.5125 - val_loss: 0.9618 - val_accuracy: 0.3000 - val_f1_m: 0.2500
Epoch 3/100
5/5 [==============================] - 0s 62ms/step - loss: 0.7709 - accuracy: 0.5437 - f1_m: 0.5437 - val_loss: 1.1613 - val_accuracy: 0.3000 - val_f1_m: 0.2500
Epoch 4/100
5/5 [==============================] - 0s 65ms/step - loss: 0.8278 - accuracy: 0.5250 - f1_m: 0.5250 - val_loss: 1.1467 - val_accuracy: 0.3000 - val_f1_m: 0.2500
Epoch 5/100
5/5 [==============================] - 0s 61ms/step - loss: 0.7264 - accuracy: 0.5500 - f1_m: 0.5500 - val_loss: 1.3189 - val_accuracy: 0.3000 - val_f1_m: 0.2500
Epoch 6/100
5/5 [==============================] - 0s 64ms/step - loss: 0.6259 - accuracy: 0.7000 - f1_m: 0.7000 - val_loss: 1.17

### Temperature

In [ ]:
size=1
X=df['data']['TEMP'].reshape(-1,10,128,size)
Y=df['labels']['arousal']

model_work(1)

Epoch 1/100
5/5 [==============================] - 1s 240ms/step - loss: 1.3529 - accuracy: 0.5000 - f1_m: 0.5000 - val_loss: 0.6875 - val_accuracy: 0.7125 - val_f1_m: 0.7604
Epoch 2/100
5/5 [==============================] - 0s 63ms/step - loss: 0.8726 - accuracy: 0.4812 - f1_m: 0.4812 - val_loss: 0.7926 - val_accuracy: 0.3000 - val_f1_m: 0.2500
Epoch 3/100
5/5 [==============================] - 0s 64ms/step - loss: 0.8892 - accuracy: 0.5500 - f1_m: 0.5500 - val_loss: 0.7893 - val_accuracy: 0.3000 - val_f1_m: 0.2500
Epoch 4/100
5/5 [==============================] - 0s 62ms/step - loss: 0.8465 - accuracy: 0.5063 - f1_m: 0.5062 - val_loss: 0.8410 - val_accuracy: 0.3000 - val_f1_m: 0.2500
Epoch 5/100
5/5 [==============================] - 0s 65ms/step - loss: 0.7629 - accuracy: 0.5813 - f1_m: 0.5812 - val_loss: 0.8977 - val_accuracy: 0.3000 - val_f1_m: 0.2500
Epoch 6/100
5/5 [==============================] - 0s 63ms/step - loss: 0.7422 - accuracy: 0.6000 - f1_m: 0.6000 - val_loss: 0.81

### GSR+PPG

In [ ]:
size=2
X=np.concatenate((df['data']['GSR'].reshape(-1,10,128,1),df['data']['PPG'].reshape(-1,10,128,1)),axis=3)
Y=df['labels']['arousal']

model_work(size)

Epoch 1/100
5/5 [==============================] - 1s 238ms/step - loss: 1.1407 - accuracy: 0.5125 - f1_m: 0.5125 - val_loss: 0.6506 - val_accuracy: 0.7000 - val_f1_m: 0.7500
Epoch 2/100
5/5 [==============================] - 0s 64ms/step - loss: 0.8885 - accuracy: 0.5500 - f1_m: 0.5500 - val_loss: 0.6480 - val_accuracy: 0.7000 - val_f1_m: 0.7500
Epoch 3/100
5/5 [==============================] - 0s 65ms/step - loss: 0.7244 - accuracy: 0.6187 - f1_m: 0.6187 - val_loss: 0.6720 - val_accuracy: 0.7000 - val_f1_m: 0.7500
Epoch 4/100
5/5 [==============================] - 0s 63ms/step - loss: 0.7981 - accuracy: 0.6125 - f1_m: 0.6125 - val_loss: 0.7323 - val_accuracy: 0.3250 - val_f1_m: 0.2708
Epoch 5/100
5/5 [==============================] - 0s 66ms/step - loss: 0.6629 - accuracy: 0.6562 - f1_m: 0.6562 - val_loss: 0.6257 - val_accuracy: 0.7000 - val_f1_m: 0.7500
Epoch 6/100
5/5 [==============================] - 0s 65ms/step - loss: 0.6740 - accuracy: 0.6938 - f1_m: 0.6937 - val_loss: 0.68

### GSR+Temperature

In [ ]:
X=np.concatenate((df['data']['GSR'].reshape(-1,10,128,1),df['data']['TEMP'].reshape(-1,10,128,1)),axis=3)
Y=df['labels']['arousal']
model_work(size)

Epoch 1/100
5/5 [==============================] - 1s 239ms/step - loss: 1.1277 - accuracy: 0.5437 - f1_m: 0.5437 - val_loss: 0.6147 - val_accuracy: 0.7000 - val_f1_m: 0.7500
Epoch 2/100
5/5 [==============================] - 0s 66ms/step - loss: 0.9117 - accuracy: 0.6438 - f1_m: 0.6437 - val_loss: 0.6334 - val_accuracy: 0.7000 - val_f1_m: 0.7500
Epoch 3/100
5/5 [==============================] - 0s 61ms/step - loss: 0.8704 - accuracy: 0.6125 - f1_m: 0.6125 - val_loss: 0.6645 - val_accuracy: 0.7125 - val_f1_m: 0.7604
Epoch 4/100
5/5 [==============================] - 0s 63ms/step - loss: 0.9254 - accuracy: 0.5250 - f1_m: 0.5250 - val_loss: 0.7091 - val_accuracy: 0.3000 - val_f1_m: 0.2500
Epoch 5/100
5/5 [==============================] - 0s 64ms/step - loss: 0.7098 - accuracy: 0.6062 - f1_m: 0.6062 - val_loss: 0.6316 - val_accuracy: 0.7250 - val_f1_m: 0.7708
Epoch 6/100
5/5 [==============================] - 0s 62ms/step - loss: 0.7661 - accuracy: 0.6250 - f1_m: 0.6250 - val_loss: 0.78

### PPG+Temperature

In [ ]:
X=np.concatenate((df['data']['PPG'].reshape(-1,10,128,1),df['data']['TEMP'].reshape(-1,10,128,1)),axis=3)
Y=df['labels']['arousal']
model_work(size)

Epoch 1/100
5/5 [==============================] - 1s 247ms/step - loss: 1.1798 - accuracy: 0.5750 - f1_m: 0.5750 - val_loss: 0.8756 - val_accuracy: 0.3000 - val_f1_m: 0.2500
Epoch 2/100
5/5 [==============================] - 0s 66ms/step - loss: 0.9276 - accuracy: 0.4938 - f1_m: 0.4937 - val_loss: 0.8016 - val_accuracy: 0.3000 - val_f1_m: 0.2500
Epoch 3/100
5/5 [==============================] - 0s 65ms/step - loss: 0.9397 - accuracy: 0.5312 - f1_m: 0.5312 - val_loss: 0.8558 - val_accuracy: 0.3000 - val_f1_m: 0.2500
Epoch 4/100
5/5 [==============================] - 0s 63ms/step - loss: 0.7450 - accuracy: 0.6187 - f1_m: 0.6187 - val_loss: 0.9312 - val_accuracy: 0.3000 - val_f1_m: 0.2500
Epoch 5/100
5/5 [==============================] - 0s 68ms/step - loss: 0.7752 - accuracy: 0.6125 - f1_m: 0.6125 - val_loss: 0.8156 - val_accuracy: 0.3000 - val_f1_m: 0.2500
Epoch 6/100
5/5 [==============================] - 0s 63ms/step - loss: 0.7239 - accuracy: 0.5938 - f1_m: 0.5937 - val_loss: 1.06

### All

In [ ]:
size=3
X=np.concatenate((df['data']['GSR'].reshape(-1,10,128,1), df['data']['PPG'].reshape(-1,10,128,1),df['data']['TEMP'].reshape(-1,10,128,1)),axis=3)
Y=df['labels']['arousal']
model_work(size)

Epoch 1/100
5/5 [==============================] - 1s 241ms/step - loss: 1.4758 - accuracy: 0.4688 - f1_m: 0.4687 - val_loss: 0.8343 - val_accuracy: 0.3000 - val_f1_m: 0.2500
Epoch 2/100
5/5 [==============================] - 0s 63ms/step - loss: 0.9520 - accuracy: 0.4875 - f1_m: 0.4875 - val_loss: 0.9161 - val_accuracy: 0.3000 - val_f1_m: 0.2500
Epoch 3/100
5/5 [==============================] - 0s 64ms/step - loss: 0.8052 - accuracy: 0.5625 - f1_m: 0.5625 - val_loss: 0.8197 - val_accuracy: 0.3000 - val_f1_m: 0.2500
Epoch 4/100
5/5 [==============================] - 0s 64ms/step - loss: 0.8424 - accuracy: 0.5813 - f1_m: 0.5812 - val_loss: 0.8502 - val_accuracy: 0.3000 - val_f1_m: 0.2500
Epoch 5/100
5/5 [==============================] - 0s 62ms/step - loss: 0.6696 - accuracy: 0.6062 - f1_m: 0.6062 - val_loss: 0.8435 - val_accuracy: 0.3000 - val_f1_m: 0.2500
Epoch 6/100
5/5 [==============================] - 0s 65ms/step - loss: 0.6205 - accuracy: 0.6687 - f1_m: 0.6687 - val_loss: 0.83

## Valence

### GSR

In [ ]:
size=1
X=df['data']['GSR'].reshape(-1,10,128,size)
Y=df['labels']['valence']

model_work(size)

Epoch 1/100
5/5 [==============================] - 1s 241ms/step - loss: 1.1540 - accuracy: 0.5250 - f1_m: 0.5250 - val_loss: 0.7221 - val_accuracy: 0.4750 - val_f1_m: 0.5625
Epoch 2/100
5/5 [==============================] - 0s 61ms/step - loss: 1.0932 - accuracy: 0.5875 - f1_m: 0.5875 - val_loss: 0.7542 - val_accuracy: 0.4750 - val_f1_m: 0.5625
Epoch 3/100
5/5 [==============================] - 0s 63ms/step - loss: 1.0028 - accuracy: 0.5312 - f1_m: 0.5312 - val_loss: 0.8743 - val_accuracy: 0.4750 - val_f1_m: 0.5625
Epoch 4/100
5/5 [==============================] - 0s 63ms/step - loss: 0.8230 - accuracy: 0.6000 - f1_m: 0.6000 - val_loss: 0.8548 - val_accuracy: 0.4750 - val_f1_m: 0.5625
Epoch 5/100
5/5 [==============================] - 0s 62ms/step - loss: 0.9588 - accuracy: 0.5500 - f1_m: 0.5500 - val_loss: 0.9395 - val_accuracy: 0.4750 - val_f1_m: 0.5625
Epoch 6/100
5/5 [==============================] - 0s 61ms/step - loss: 0.8485 - accuracy: 0.5813 - f1_m: 0.5812 - val_loss: 0.91

###PPG

In [ ]:
X=df['data']['PPG'].reshape(-1,10,128,size)
Y=df['labels']['valence']

model_work(size)

Epoch 1/100
5/5 [==============================] - 1s 239ms/step - loss: 1.3207 - accuracy: 0.4812 - f1_m: 0.4812 - val_loss: 0.7120 - val_accuracy: 0.5250 - val_f1_m: 0.4375
Epoch 2/100
5/5 [==============================] - 0s 63ms/step - loss: 0.9030 - accuracy: 0.5000 - f1_m: 0.5000 - val_loss: 0.7132 - val_accuracy: 0.5250 - val_f1_m: 0.4375
Epoch 3/100
5/5 [==============================] - 0s 66ms/step - loss: 0.8194 - accuracy: 0.5312 - f1_m: 0.5312 - val_loss: 0.7053 - val_accuracy: 0.5000 - val_f1_m: 0.4167
Epoch 4/100
5/5 [==============================] - 0s 61ms/step - loss: 0.8103 - accuracy: 0.5063 - f1_m: 0.5062 - val_loss: 0.7100 - val_accuracy: 0.4125 - val_f1_m: 0.4792
Epoch 5/100
5/5 [==============================] - 0s 63ms/step - loss: 0.8719 - accuracy: 0.5312 - f1_m: 0.5312 - val_loss: 0.7073 - val_accuracy: 0.4000 - val_f1_m: 0.4583
Epoch 6/100
5/5 [==============================] - 0s 63ms/step - loss: 0.8839 - accuracy: 0.4875 - f1_m: 0.4875 - val_loss: 0.71

### Temperature

In [ ]:
X=df['data']['TEMP'].reshape(-1,10,128,size)
Y=df['labels']['valence']

model_work(size)

Epoch 1/100
5/5 [==============================] - 1s 243ms/step - loss: 0.9686 - accuracy: 0.5562 - f1_m: 0.5562 - val_loss: 0.7015 - val_accuracy: 0.5250 - val_f1_m: 0.4375
Epoch 2/100
5/5 [==============================] - 0s 66ms/step - loss: 0.9321 - accuracy: 0.5625 - f1_m: 0.5625 - val_loss: 0.7003 - val_accuracy: 0.5125 - val_f1_m: 0.4271
Epoch 3/100
5/5 [==============================] - 0s 63ms/step - loss: 1.0237 - accuracy: 0.4688 - f1_m: 0.4687 - val_loss: 0.7119 - val_accuracy: 0.4000 - val_f1_m: 0.4792
Epoch 4/100
5/5 [==============================] - 0s 64ms/step - loss: 0.9846 - accuracy: 0.5312 - f1_m: 0.5312 - val_loss: 0.7020 - val_accuracy: 0.5125 - val_f1_m: 0.5000
Epoch 5/100
5/5 [==============================] - 0s 62ms/step - loss: 1.0804 - accuracy: 0.4688 - f1_m: 0.4687 - val_loss: 0.7033 - val_accuracy: 0.4500 - val_f1_m: 0.4687
Epoch 6/100
5/5 [==============================] - 0s 60ms/step - loss: 0.8499 - accuracy: 0.5500 - f1_m: 0.5500 - val_loss: 0.70

### GSR+PPG

In [ ]:
size=2
X=np.concatenate((df['data']['GSR'].reshape(-1,10,128,1),df['data']['PPG'].reshape(-1,10,128,1)),axis=3)
Y=df['labels']['valence']
model_work(size)

Epoch 1/100
5/5 [==============================] - 1s 244ms/step - loss: 1.1452 - accuracy: 0.5000 - f1_m: 0.5000 - val_loss: 0.7183 - val_accuracy: 0.4750 - val_f1_m: 0.5625
Epoch 2/100
5/5 [==============================] - 0s 63ms/step - loss: 0.8283 - accuracy: 0.6250 - f1_m: 0.6250 - val_loss: 0.7215 - val_accuracy: 0.4750 - val_f1_m: 0.5625
Epoch 3/100
5/5 [==============================] - 0s 62ms/step - loss: 0.7525 - accuracy: 0.6375 - f1_m: 0.6375 - val_loss: 0.7358 - val_accuracy: 0.4750 - val_f1_m: 0.5625
Epoch 4/100
5/5 [==============================] - 0s 63ms/step - loss: 0.7772 - accuracy: 0.5938 - f1_m: 0.5937 - val_loss: 0.7660 - val_accuracy: 0.4750 - val_f1_m: 0.5625
Epoch 5/100
5/5 [==============================] - 0s 65ms/step - loss: 0.8350 - accuracy: 0.6187 - f1_m: 0.6187 - val_loss: 0.7510 - val_accuracy: 0.4750 - val_f1_m: 0.5625
Epoch 6/100
5/5 [==============================] - 0s 62ms/step - loss: 0.7533 - accuracy: 0.6438 - f1_m: 0.6437 - val_loss: 0.77

### GSR+Temperature

In [ ]:
X=np.concatenate((df['data']['GSR'].reshape(-1,10,128,1),df['data']['TEMP'].reshape(-1,10,128,1)),axis=3)
Y=df['labels']['valence']
model_work(size)


Epoch 1/100
5/5 [==============================] - 1s 240ms/step - loss: 1.2158 - accuracy: 0.5750 - f1_m: 0.5750 - val_loss: 0.7204 - val_accuracy: 0.4750 - val_f1_m: 0.5625
Epoch 2/100
5/5 [==============================] - 0s 64ms/step - loss: 1.0807 - accuracy: 0.5375 - f1_m: 0.5375 - val_loss: 0.6885 - val_accuracy: 0.5250 - val_f1_m: 0.4375
Epoch 3/100
5/5 [==============================] - 0s 64ms/step - loss: 0.9507 - accuracy: 0.5562 - f1_m: 0.5562 - val_loss: 0.7039 - val_accuracy: 0.4750 - val_f1_m: 0.5625
Epoch 4/100
5/5 [==============================] - 0s 65ms/step - loss: 0.7800 - accuracy: 0.5938 - f1_m: 0.5937 - val_loss: 0.7041 - val_accuracy: 0.5250 - val_f1_m: 0.4375
Epoch 5/100
5/5 [==============================] - 0s 62ms/step - loss: 0.8204 - accuracy: 0.5938 - f1_m: 0.5937 - val_loss: 0.6995 - val_accuracy: 0.5000 - val_f1_m: 0.4375
Epoch 6/100
5/5 [==============================] - 0s 61ms/step - loss: 0.7896 - accuracy: 0.6187 - f1_m: 0.6187 - val_loss: 0.70

### PPG+Temperature

In [ ]:
X=np.concatenate((df['data']['PPG'].reshape(-1,10,128,1),df['data']['TEMP'].reshape(-1,10,128,1)),axis=3)
Y=df['labels']['valence']
model_work(size)

Epoch 1/100
5/5 [==============================] - 1s 243ms/step - loss: 1.2781 - accuracy: 0.4875 - f1_m: 0.4875 - val_loss: 0.7079 - val_accuracy: 0.5250 - val_f1_m: 0.4375
Epoch 2/100
5/5 [==============================] - 0s 62ms/step - loss: 1.1232 - accuracy: 0.5250 - f1_m: 0.5250 - val_loss: 0.7990 - val_accuracy: 0.5250 - val_f1_m: 0.4375
Epoch 3/100
5/5 [==============================] - 0s 63ms/step - loss: 0.9221 - accuracy: 0.5437 - f1_m: 0.5437 - val_loss: 0.7243 - val_accuracy: 0.5250 - val_f1_m: 0.4375
Epoch 4/100
5/5 [==============================] - 0s 62ms/step - loss: 0.8193 - accuracy: 0.5375 - f1_m: 0.5375 - val_loss: 0.7167 - val_accuracy: 0.5250 - val_f1_m: 0.4375
Epoch 5/100
5/5 [==============================] - 0s 65ms/step - loss: 0.8540 - accuracy: 0.6250 - f1_m: 0.6250 - val_loss: 0.6929 - val_accuracy: 0.5250 - val_f1_m: 0.4375
Epoch 6/100
5/5 [==============================] - 0s 64ms/step - loss: 0.8039 - accuracy: 0.5875 - f1_m: 0.5875 - val_loss: 0.69

### All

In [ ]:
size=3
X=np.concatenate((df['data']['GSR'].reshape(-1,10,128,1), df['data']['PPG'].reshape(-1,10,128,1),df['data']['TEMP'].reshape(-1,10,128,1)),axis=3)
Y=df['labels']['valence']
model_work(size)

Epoch 1/100
5/5 [==============================] - 1s 241ms/step - loss: 1.2956 - accuracy: 0.4812 - f1_m: 0.4812 - val_loss: 0.6935 - val_accuracy: 0.5500 - val_f1_m: 0.4687
Epoch 2/100
5/5 [==============================] - 0s 62ms/step - loss: 0.9800 - accuracy: 0.5500 - f1_m: 0.5500 - val_loss: 0.7037 - val_accuracy: 0.4125 - val_f1_m: 0.3437
Epoch 3/100
5/5 [==============================] - 0s 64ms/step - loss: 1.0422 - accuracy: 0.4437 - f1_m: 0.4437 - val_loss: 0.7208 - val_accuracy: 0.4750 - val_f1_m: 0.5625
Epoch 4/100
5/5 [==============================] - 0s 64ms/step - loss: 0.7224 - accuracy: 0.5938 - f1_m: 0.5937 - val_loss: 0.7175 - val_accuracy: 0.4750 - val_f1_m: 0.5625
Epoch 5/100
5/5 [==============================] - 0s 63ms/step - loss: 0.7369 - accuracy: 0.5625 - f1_m: 0.5625 - val_loss: 0.7208 - val_accuracy: 0.4750 - val_f1_m: 0.5625
Epoch 6/100
5/5 [==============================] - 0s 62ms/step - loss: 0.7186 - accuracy: 0.6000 - f1_m: 0.6000 - val_loss: 0.72

# Leave one subject out

In [ ]:
from sklearn.model_selection import LeaveOneOut

loo = LeaveOneOut()

##LSTM

In [ ]:
import numpy as np
X_GSR=[]
X_PPG=[]
X_TEMP=[]

Y_A=[]
Y_V=[]
for id in dataset:
    X_GSR.append(dataset[id]['data']['GSR'])
    X_PPG.append(dataset[id]['data']['PPG'])
    X_TEMP.append(dataset[id]['data']['TEMP'])
    Y_A.append(dataset[id]['labels']['arousal'])
    Y_V.append(dataset[id]['labels']['valence'])
    
X_GSR=np.array(X_GSR)
X_PPG=np.array(X_PPG)
X_TEMP=np.array(X_TEMP)
Y_A=np.array(Y_A)
Y_V=np.array(Y_V)



In [ ]:
def model_work_L(size,dataset,X,Y):
    model = Sequential()
    model.add(Bidirectional(LSTM(128, return_sequences=True, dropout=0.2, recurrent_dropout=0.2), input_shape=(10,128*size)))
    model.add(BatchNormalization())
    model.add(Bidirectional(LSTM(128, dropout=0.2, recurrent_dropout=0.2)))
    model.add(BatchNormalization())

    model.add(Dense(2, activation='softmax'))
    
    acc_list=[]
    f_list=[]
    for train_index, test_index in loo.split(dataset):
        if test_index<=23:
            continue
        print("TRAIN:", train_index, "TEST:", test_index)
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = Y[train_index], Y[test_index]

        X_train=X_train.reshape(-1,10,128)
        X_test=X_test.reshape(-1,10,128)
        y_train=y_train.reshape(-1,2)
        y_test=y_test.reshape(-1,2)

        model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy",f1_m])
        early_stopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
        model.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test), callbacks=[early_stopping_cb])
        _,acc,f=model.evaluate(X_test,y_test)
        print(acc,f)
        acc_list.append(acc * 100)
        f_list.append(f * 100)
    return acc_list,f_list


### Arousal

#### GSR

In [ ]:
size=1
acc_A_G, f_A_G = model_work_L(1,dataset,X_GSR,Y_A)
print(acc_A_G)
print(f_A_G)

TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 28 29 30 31] TEST: [27]
Epoch 1/100
233/233 [==============================] - 51s 217ms/step - loss: 0.7936 - accuracy: 0.5380 - f1_m: 0.5382 - val_loss: 0.7104 - val_accuracy: 0.5000 - val_f1_m: 0.5078
Epoch 2/100
233/233 [==============================] - 50s 216ms/step - loss: 0.7120 - accuracy: 0.5470 - f1_m: 0.5472 - val_loss: 0.7382 - val_accuracy: 0.5000 - val_f1_m: 0.5078
Epoch 3/100
233/233 [==============================] - 50s 216ms/step - loss: 0.6953 - accuracy: 0.5626 - f1_m: 0.5624 - val_loss: 0.7306 - val_accuracy: 0.5042 - val_f1_m: 0.5117
Epoch 4/100
233/233 [==============================] - 50s 215ms/step - loss: 0.6963 - accuracy: 0.5586 - f1_m: 0.5586 - val_loss: 0.7238 - val_accuracy: 0.4958 - val_f1_m: 0.5039
Epoch 5/100
233/233 [==============================] - 50s 216ms/step - loss: 0.6919 - accuracy: 0.5640 - f1_m: 0.5638 - val_loss: 0.7273 - val_accuracy: 0.4958 - val

#### PPG

In [ ]:
acc_A_P, f_A_P = model_work_L(size,dataset,X_PPG,Y_A)
print(acc_A_P)
print(f_A_P)

ValueError: ignored

#### Temperature

In [ ]:
acc_A_T, f_A_T = model_work_L(size,dataset,X_TEMP,Y_A)
print(acc_A_T)
print(f_A_T)

### Valence

#### GSR

In [ ]:
size=1
acc_V_G, f_V_G = model_work_L(1,dataset,X_GSR,Y_V)
print(acc_V_G)
print(f_V_G)

TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 25 26 27 28 29 30 31] TEST: [24]
Epoch 1/100
233/233 [==============================] - 33s 142ms/step - loss: 0.7871 - accuracy: 0.5266 - f1_m: 0.5260 - val_loss: 0.7848 - val_accuracy: 0.4875 - val_f1_m: 0.4570
Epoch 2/100
233/233 [==============================] - 33s 140ms/step - loss: 0.7163 - accuracy: 0.5267 - f1_m: 0.5264 - val_loss: 0.6815 - val_accuracy: 0.5750 - val_f1_m: 0.5547
Epoch 3/100
233/233 [==============================] - 32s 139ms/step - loss: 0.7057 - accuracy: 0.5348 - f1_m: 0.5349 - val_loss: 0.7381 - val_accuracy: 0.4625 - val_f1_m: 0.4336
Epoch 4/100
233/233 [==============================] - 34s 145ms/step - loss: 0.7106 - accuracy: 0.5415 - f1_m: 0.5413 - val_loss: 0.6939 - val_accuracy: 0.6125 - val_f1_m: 0.6094
Epoch 5/100
233/233 [==============================] - 34s 147ms/step - loss: 0.7013 - accuracy: 0.5452 - f1_m: 0.5453 - val_loss: 0.6720 - val_accuracy: 0.6125 - val

## CRNN

In [ ]:
def model_work(size,dataset,X,Y):
    model = Sequential()
    model.add(Conv2D(64, (3, 3), padding='same', name='conv1', kernel_initializer='he_normal', input_shape=(10, 128, size)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), name='max1'))

    model.add(Conv2D(128, (3, 3), padding='same', name='conv2', kernel_initializer='he_normal'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))



    model.add(keras.layers.Reshape((10,32*128,) ))
    model.add(Dense(64, activation='relu', kernel_initializer='he_normal', name='dense1'))
    model.add(Bidirectional(LSTM(128, return_sequences=True, dropout=0.2, recurrent_dropout=0.2), input_shape=(10,128)))
    model.add(BatchNormalization())
    model.add(Bidirectional(LSTM(128, dropout=0.2, recurrent_dropout=0.2)))
    model.add(BatchNormalization())

    model.add(Dense(2, activation='softmax'))

    
    acc_list=[]
    f_list=[]
    for train_index, test_index in loo.split(dataset):
        if test_index<=4:
            continue
        print("TRAIN:", train_index, "TEST:", test_index)
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = Y[train_index], Y[test_index]

        X_train=X_train.reshape(-1,10,128,size)
        X_test=X_test.reshape(-1,10,128,size)
        y_train=y_train.reshape(-1,2)
        y_test=y_test.reshape(-1,2)

        model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy",f1_m])
        early_stopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
        model.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test), callbacks=[early_stopping_cb])
        _,acc,f=model.evaluate(X_test,y_test)
        print(acc,f)
        acc_list.append(acc * 100)
        f_list.append(f * 100)
    return acc_list,f_list


### Arousal

#### GSR

In [ ]:
size=1
acc_A_G2, f_A_G2 = model_work(size,dataset,X_GSR,Y_A)
print(acc_A_G2)
print(f_A_G2)

TRAIN: [ 0  1  2  3  4  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31] TEST: [5]
Epoch 1/100
233/233 [==============================] - 76s 325ms/step - loss: 0.7570 - accuracy: 0.5477 - f1_m: 0.5476 - val_loss: 0.7715 - val_accuracy: 0.4167 - val_f1_m: 0.4258
Epoch 2/100
233/233 [==============================] - 74s 320ms/step - loss: 0.7052 - accuracy: 0.5563 - f1_m: 0.5566 - val_loss: 0.7950 - val_accuracy: 0.4083 - val_f1_m: 0.4141
Epoch 3/100
233/233 [==============================] - 74s 320ms/step - loss: 0.6963 - accuracy: 0.5624 - f1_m: 0.5622 - val_loss: 0.8942 - val_accuracy: 0.4125 - val_f1_m: 0.4258
Epoch 4/100
233/233 [==============================] - 75s 322ms/step - loss: 0.6914 - accuracy: 0.5696 - f1_m: 0.5696 - val_loss: 0.7937 - val_accuracy: 0.4292 - val_f1_m: 0.4414
Epoch 5/100
233/233 [==============================] - 75s 323ms/step - loss: 0.6926 - accuracy: 0.5770 - f1_m: 0.5769 - val_loss: 0.8184 - val_accuracy: 0.3917 - val_

#### PPG

In [ ]:
acc_A_T2, f_A_T2 = model_work(size,dataset,X_TEMP,Y_A)
print(acc_A_T2)
print(f_A_T2)

#### Temperature

In [ ]:
acc_A_T2, f_A_T2 = model_work(size,dataset,X_TEMP,Y_A)
print(acc_A_T2)
print(f_A_T2)

### Valence

#### GSR

In [ ]:
size=1
acc_A_V2, f_A_V2 = model_work(size,dataset,X_GSR,Y_V)
print(acc_A_V2)
print(f_A_V2)